# Spatial Batch Normalization

When using batch normalization with PyTorch, you should be paying attention to the number of dimensions in the input (see <a href="https://pytorch.org/docs/stable/nn.html#batchnorm1d">BatchNorm1d</a>, <a href="https://pytorch.org/docs/stable/nn.html#batchnorm2d">BatchNorm2d</a> etc.)

We already saw that batch normalization is a very useful technique for training deep fully-connected networks. Batch normalization can also be used for convolution networks, but we need to tweak it a bit; the modification will be called "spatial batch normalization". 

# 1. Extension from Batch Normalization

Normally batch-normalization accepts inputs of shape $(N, D)$ and produces outputs of shape $(N, D)$, where we normalize across the mini-batch dimension $N$. For data coming from convolution layers, batch normalization needs to accept inputs of shape $(N, C, H, W)$ and produce outputs of shape $(N, C, H, W)$ where the $N$ dimension gives the mini-batch size and the $(H, W)$ dimensions give the spatial size of the feature map.

If the feature map was produced using convolutions, we apply the same filter to different locations of feature maps from last layer and to the whole batch of data to get a single feature channel. Then we expect the statistics of each feature channel to be relatively consistent both between different images and different locations within the same image. Therefore spatial batch normalization computes a mean and variance for each of the $C$ feature channels by computing statistics over both the mini-batch dimension $N$ and the spatial dimensions $H$ and $W$.

For a better understanding of relationship and difference between batch normalization and spatial batch normalization, the picture taken from [CS231n Note](http://cs231n.stanford.edu/slides/2018/cs231n_2018_lecture07.pdf) gives us a comparison.

![](https://drive.google.com/uc?export=view&id=1l4qVJQWQ2iSogVY3euZ7Rt9osbGmGNmO)

Basically they share the same computation rules, i.e. normalize over some dimensions and transform to new output based on $y = \gamma (x - \mu) / \delta + \beta$. But they operate in different dimensions, since images are stored in a higher dimension tensor.

## (Optional) Mount folder in Colab

Uncomment thefollowing cell to mount your gdrive if you are using the notebook in google colab:

In [ ]:
# Use the following lines if you want to use Google Colab
# We presume you created a folder "DL_homeworks" within your main drive folder, and put the homework there.
# NOTE: terminate all other colab sessions that use GPU!
# NOTE 2: Make sure the correct homework folder (e.g homework_08) is given.


from google.colab import drive
import os

gdrive_path='/content/gdrive/MyDrive/DL_homeworks/homework_08'

# This will mount your google drive under 'MyDrive'
drive.mount('/content/gdrive', force_remount=True)
# In order to access the files in this notebook we have to navigate to the correct folder
os.chdir(gdrive_path)
# Check manually if all files are present
print(sorted(os.listdir()))

# Relevant packages. NOTE: Be patient, it might take a few minutes.
!python -m pip install tensorboard==2.8.0 pytorch-lightning==1.6.0 torchtext


# 2. Implementation

## 2.1 Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

from exercise_code.layers import (
    spatial_batchnorm_forward, 
    spatial_batchnorm_backward,
)
from exercise_code.tests.gradient_check import (
    eval_numerical_gradient_array,
    eval_numerical_gradient,
    rel_error,
)
from exercise_code.tests.spatial_batchnorm_tests import (
    test_spatial_batchnorm_forward,
    test_spatial_batchnorm_backward,
)

from exercise_code.networks.SpatialBatchNormModel import (
    SimpleNetwork,
    SpatialBatchNormNetwork,
)

%load_ext autoreload
%autoreload 2

# supress cluttering warnings in solutions
import warnings
warnings.filterwarnings('ignore')

## 2.2 Spatial Batch Normalization: Forward

<div class="alert alert-info">
    <h3>Task: Implement</h3>
    <p>In the file <code>exercise_code/layers.py </code>, implement the forward pass for spatial batch normalization in the function <code>spatial_batchnorm_forward</code>. Check your implementation by running the following cell:
 </p>
    <p>
    <b>Hints</b>: Be careful about the difference of dimensions between batch normalization and spatial batch normalization.
    </p>
</div>

In [2]:
test_spatial_batchnorm_forward()

SpatialBatchnormForwardTest with trivial beta and gamma (train) passed.
SpatialBatchnormForwardTest with nontrivial beta and gamma (train) passed.
SpatialBatchnormForwardTest with trivial beta and gamma (test) passed.
All tests passed for your spatial batchnorm implementation. Tests passed: 3/3


## 2.3 Spatial Batch Normalization: backward

Now that you have successfully implemented the spatial batch normalization forward pass by using the batch normalization functions, it would be easy and straightforward to finish the backward pass.

<div class="alert alert-info">
    <h3>Task: Implement</h3>
    <p>In the file <code>exercise_code/layers.py</code>, implement the backward pass for batch normalization, alternative batch normalization, spatial batch normalization in the functions <code>batchnorm_backward</code>, <code>batchnorm_backward_alt</code>, <code>spatial_batchnorm_backward</code>. Run the following to check your implementation using a numeric gradient check:
 
</div>


In [3]:
test_spatial_batchnorm_backward()

SpatialBatchnormBackwardTest passed.


## 2.4 Spatial Batch Normalization in Pytorch Lightning

Similar as the batch normalization task from previous homework, here we would also like to do some experiments using Pytorch Lightning to see the effect of spatial batch normalization.

### 2.4.1 Setup TensorBoard

After some experience with TensorBoard so far, TensorBoard should be your friend in tuning your network and monitoring the training process. Throughout this notebook, feel free to add further logs or visualizations your TensorBoard!

In [4]:
# Few Hyperparameters before we start things off
batch_size = 50

logdir = './lightning_logs'
if os.path.exists(logdir):
    # We delete the logs on the first run
    shutil.rmtree(logdir)
os.mkdir(logdir)

epochs = 5
learning_rate = 0.0005

In [5]:
%load_ext tensorboard
!/Users/tigrangaplanyan/anaconda3/envs/Homeworks/bin/tensorboard --logdir=logdir

TensorFlow installation not found - running with reduced feature set.
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.9.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C


#### Reminder: Run Tensorboard

<div class="alert alert-success">
    <p> 
    There are a few ways to run tensorboard: <br><br>
    1) Use the cell above. <br>
    2) Press <code> Ctrl + Shift + p </code> and look for <code> Tensorboard </code> <br>
    3) Put <code> tensorboard --logdir=logdir </code> in the terminal.

</div>

### 2.4.2 Train a model without Spatial Batch Normalization

<div class="alert alert-success">
    <h3>Task: Check Code</h3>
    <p>We have already implemented a <code>SimpleNetwork</code> without spatial batch normalization in <code>exercise_code/networks/SpatialBatchNormModel.py</code>. Feel free to check it out and play around with the parameters. The cell below is setting up a short training process for this network.
 </p>
</div>

In [6]:
# train
model = SimpleNetwork(batch_size=batch_size, learning_rate=learning_rate)
# Creating a logging object
simple_network_logger = TensorBoardLogger(
    save_dir=logdir,
)
trainer = pl.Trainer(max_epochs=batch_size , logger=simple_network_logger, accelerator="auto")

trainer.fit(model)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ./lightning_logs/lightning_logs

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 4.8 K 
1 | fc    | Linear     | 15.7 K
-------------------------------------
20.5 K    Trainable params
0         Non-trainable params
20.5 K    Total params
0.082     Total estimated model params size (MB)


Epoch 0: 100%|█| 960/960 [00:09<00:00, 105.05it/s, v_num=0, train/loss_step=0.90
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  39%|██████▏         | 93/240 [00:00<00:00, 172.99it/s]


Validation DataLoader 0:  79%|███████████▉   | 190/240 [00:01<00:00, 173.53it/s]


Validation DataLoader 0: 100%|███████████████| 240/240 [00:01<00:00, 173.32it/s]
Epoch 1: 100%|█| 960/960 [00:09<00:00, 105.89it/s, v_num=0, train/loss_step=0.72
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  18%|██▊             | 43/240 [00:00<00:01, 167.14it/s]


Validation DataLoader 0:  58%|████████▊      | 140/240 [00:00<00:00, 166.05it/s]


...

Validation DataLoader 0: 100%|███████████████| 240/240 [00:01<00:00, 161.64it/s]
Epoch 49: 100%|█| 960/960 [00:09<00:00, 100.35it/s, v_num=0, train/loss_step=0.4
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  28%|████▌           | 68/240 [00:00<00:01, 167.38it/s]


Validation DataLoader 0:  69%|██████████▎    | 165/240 [00:00<00:00, 166.37it/s]


Validation DataLoader 0: 100%|███████████████| 240/240 [00:01<00:00, 165.68it/s]
Epoch 49: 100%|█| 960/960 [00:11<00:00, 87.06it/s, v_num=0, train/loss_step=0.46

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|█| 960/960 [00:11<00:00, 87.00it/s, v_num=0, train/loss_step=0.46


### 2.4.3 Train a model with Spatial Batch Normalization

<div class="alert alert-success">
    <h3>Task: Check Code</h3>
    <p> Now that we have already seen how our simple network should work, let us look at a model that is actually using spatial batch normalization. Again, we provide you with such a model <code>SpatialBatchNormNetwork</code> in <code>exercise_code/netowkrs/SpatialBatchNormModel.py</code>. Same as before: Feel free to check it out and play around with the parameters. The cell below is setting up a short training process for this model. 
 </p>
</div>

In [7]:
model_bn = SpatialBatchNormNetwork(batch_size=batch_size, learning_rate=learning_rate)
spatial_bn_network_logger = TensorBoardLogger(
    save_dir=logdir,
    name='spatial_bn_network'  
)

trainer = pl.Trainer(max_epochs=epochs, logger=spatial_bn_network_logger, log_every_n_steps=2, accelerator="auto")
trainer.fit(model_bn)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ./lightning_logs/spatial_bn_network

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 4.9 K 
1 | fc    | Linear     | 15.7 K
-------------------------------------
20.6 K    Trainable params
0         Non-trainable params
20.6 K    Total params
0.082     Total estimated model params size (MB)


Epoch 0: 100%|█| 960/960 [00:11<00:00, 85.62it/s, v_num=0, train/loss_step=0.760
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  39%|██████▏         | 93/240 [00:00<00:00, 149.84it/s]


Validation DataLoader 0:  79%|███████████▉   | 190/240 [00:01<00:00, 152.55it/s]


Validation DataLoader 0: 100%|███████████████| 240/240 [00:01<00:00, 154.08it/s]
Epoch 1: 100%|█| 960/960 [00:11<00:00, 84.89it/s, v_num=0, train/loss_step=0.503
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  18%|██▊             | 43/240 [00:00<00:01, 157.75it/s]


Validation DataLoader 0:  58%|████████▊      | 140/240 [00:00<00:00, 158.04it/s]


...

Validation DataLoader 0: 100%|███████████████| 240/240 [00:01<00:00, 157.36it/s]
Epoch 4: 100%|█| 960/960 [00:11<00:00, 84.29it/s, v_num=0, train/loss_step=0.309
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  36%|█████▊          | 87/240 [00:00<00:00, 157.15it/s]


Validation DataLoader 0:  77%|███████████▌   | 184/240 [00:01<00:00, 156.78it/s]


Validation DataLoader 0: 100%|███████████████| 240/240 [00:01<00:00, 157.47it/s]
Epoch 4: 100%|█| 960/960 [00:12<00:00, 74.27it/s, v_num=0, train/loss_step=0.309

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|█| 960/960 [00:12<00:00, 74.21it/s, v_num=0, train/loss_step=0.309


### 2.4.4 Observations

Take a look at TensorBoard to compare the performance of both networks:

In [10]:
%load_ext tensorboard
!/Users/tigrangaplanyan/anaconda3/envs/Homeworks/bin/tensorboard --logdir lightning_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
TensorFlow installation not found - running with reduced feature set.
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.9.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C


Remember the comparison result with respect to batch normalization from the first notebook, the difference here is very similar as before, i.e. we could have lower validation loss and higher validation accuracy using spatial batch normalization. The simple experiment shows that spatial batch normalization is helpful when we use convolution networks.

# Submission Goals

- Goal 1: Successfully implement Batch Normalization and Dropout.

- Goal 2: Implement and train a convolution neural network for facial keypoint detection.
- Passing Criteria: Reach **Score >= 100**.

- Goal 3 : Successfully implement Spatial Batch Normalization

- Submission deadline: __Sunday, 26.11.2022 - 23:59__
- You can make **$\infty$** submissions until the deadline. Only your __best submission__ will be considered.

Simply run the following cell to create a zipped file for your implementations.

In [11]:
# Now zip the folder for upload
from exercise_code.util.submit import submit_exercise

submit_exercise('homework_07_08')

relevant folders: ['exercise_code', 'models']
notebooks files: ['1_BatchNormalization_Dropout.ipynb', '3_spatial_batchnorm.ipynb', '2_Facial_keypoints.ipynb']
Adding folder exercise_code
Adding folder models
Adding notebook 1_BatchNormalization_Dropout.ipynb
Adding notebook 3_spatial_batchnorm.ipynb
Adding notebook 2_Facial_keypoints.ipynb
Zipping successful! Zip is stored under: /Users/tigrangaplanyan/Downloads/DL_homeworks/homework_07_08/homework_07_08.zip
